In [1]:
from langchain.llms import OpenAI
from langchain.agents import AgentType, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.tools import Tool, StructuredTool
import json, random
import os, time
from datetime import datetime
import pytz
from coro_agent.coro_agent import initialize_coro_agent, CoroAgentSuspend
from coro_agent.coro_tools import StructuredToolWithCtx
from abc import ABC

memory = ConversationBufferMemory(memory_key='chat_history')

os.environ['OPENAI_API_KEY'] = '****'

llm = OpenAI(temperature=0)

def queryTime(action_input):
    '''
    查询此时此刻时间
    '''
    now = datetime.now()
    beijing_tz = pytz.timezone('Asia/Shanghai')
    beijing_time = now.astimezone(beijing_tz)
    return beijing_time.strftime("%Y-%m-%d %H:%M:%S")

def askUser(action_input: str):
    '''
    当你无法确定可以调用哪个工具或者调用工具缺少必要参数的时候，你可以询问用户获取想要的答案，如果用户回答不满足，你可以重复提问，输入为:[你想了解的问题]
    '''    
    return CoroAgentSuspend(output=action_input)

def searchBill(action_input: str, context:dict ):
    '''
    用于查询指定日期账单, 查询参数不可编造，如果无法从用户提问中得知，需要向用户询问，需要的参数为json格式:
    {{
        "type": "账单类型:日账单|月账单", 
        "startDate": "开始日期, 如果是日账单需要精确到日,eg:20230627, 如果是月账单只需要精确到月,eg:202306", 
        "endDate": "结束日期,格式和startDate一致"
    }}
    '''
    params = json.loads(action_input)
    return '查询到{}的账单如下：账单类型:{}\t日期:{}-{}\t 消费200元' .format(
        context['name'],
        params['type'],
        params['startDate'],
        params['endDate'],      
    )

tools = [
    StructuredToolWithCtx.from_function(searchBill),
    StructuredTool.from_function(askUser),
    StructuredTool.from_function(queryTime),
]

agent_chain = initialize_coro_agent(tools, llm, agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory, tool_execute_context={'name': 'wyw'})

r = agent_chain.run(input='我想查账单')

print(f'>>>>>>>>{r}')

0.0.295


> Entering new CoroAgentExecutor chain...
[ChatMessage(content='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.\n\

In [2]:
r = agent_chain.run(input='查询上个月的账单')
print(f'>>>>>>>>{r}')




> Entering new CoroAgentExecutor chain...
[ChatMessage(content='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.\n\nOverall

In [3]:
r = agent_chain.run(input='这个月的呢')
print(f'>>>>>>>>{r}')



> Entering new CoroAgentExecutor chain...
[ChatMessage(content='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.\n\nOverall

In [ ]:
r = agent_chain.run(input='今天的账单呢')
print(f'>>>>>>>>{r}')